In [ ]:
import gc
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
!pip install transformers
from transformers import RobertaTokenizer, RobertaConfig, TFRobertaPreTrainedModel
from transformers import TFRobertaMainLayer
#from tf.keras import get_initializer

In [ ]:
def get_initializer(initializer_range=0.02):
  """Creates a `tf.initializers.truncated_normal` with the given range.

  Args:
    initializer_range: float, initializer range for stddev.

  Returns:
    TruncatedNormal initializer with stddev = `initializer_range`.
  """
  return tf.keras.initializers.TruncatedNormal(stddev=initializer_range)

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IITR/Research paper shiz/FINAL.csv')
df2 = pd.read_csv('/content/drive/MyDrive/IITR/Research paper shiz/test_data.csv')
df=df.drop(df.columns[0], axis=1)
df2=df2.drop(df2.columns[0], axis=1)
df2 = df2.rename(columns={"text":"comment_text"})
df = df.rename(columns={"Obfuscation, Intentional vagueness, Confusion":'"Obfuscation, Intentional vagueness, Confusion"'})
df

,id,comment_text,'Bandwagon',"""Misrepresentation of Someone's Position (Straw Man)""",'Appeal to authority','Appeal to fear/prejudice','Black-and-white Fallacy/Dictatorship','Causal Oversimplification','Doubt','Exaggeration/Minimisation','Flag-waving','Glittering generalities (Virtue)','Loaded Language','Name calling/Labeling',"""Obfuscation, Intentional vagueness, Confusion""",'Presenting Irrelevant Data (Red Herring)','Reductio ad hitlerum','Repetition','Slogans','Smears','Thought-terminating cliché','Whataboutism'
0,614_batch_2,If the logic of the anti-blockade protest appl...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,605_batch_2,"Before we do the teacher, start this up your b...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,380_batch_2,"42 days before I become my president, season t...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,532_batch_2,LOOK SON \nDEMOCRATS DESTROYING STATUES OF RAC...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0
4,454_batch_2,THINKING what they got\n\nSHE WIFE OR MY WHO?\...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7922,132,WHAT KIDS THINK COMMUNISM IS\n\nWHAT COMMUNISM...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7923,591_batch_2,IMPECHATE Democratic Party\n\nArchived America...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
7924,434_batch_2,LET THE FARMERS DO HIS OWN HAIR,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
7925,567_batch_2,TONY TIMPA\nKILLED BY POLICE BRUTALITY\n- AFTE...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/IITR/Research paper shiz/FINAL.csv')
df3 = pd.read_csv('/content/drive/MyDrive/IITR/TrainDataAug.csv')
# df=df.drop(df.columns[0], axis=1)
# df=df.drop(df.columns[-1], axis=1)

#df['"Bandwagon"'] = df["Bandwagon"].fillna(0)
# df['dummy'] = df[" 'Bandwagon'"]
# df = df.rename(columns={"dummy":"'Bandwagon'"})
# df = df.drop(columns=" 'Bandwagon'", axis=1)
#df['"Obfuscation, Intentional vagueness, Confusion"']=df["Obfuscation, Intentional vagueness, Confusion"]
df = df.rename(columns={"Obfuscation, Intentional vagueness, Confusion":'"Obfuscation, Intentional vagueness, Confusion"'})
df3=df3.drop(df3.columns[0], axis=1)
df=pd.concat([df,df3])
df

,id,comment_text,'Bandwagon',"""Misrepresentation of Someone's Position (Straw Man)""",'Appeal to authority','Appeal to fear/prejudice','Black-and-white Fallacy/Dictatorship','Causal Oversimplification','Doubt','Exaggeration/Minimisation','Flag-waving','Glittering generalities (Virtue)','Loaded Language','Name calling/Labeling',"""Obfuscation, Intentional vagueness, Confusion""",'Presenting Irrelevant Data (Red Herring)','Reductio ad hitlerum','Repetition','Slogans','Smears','Thought-terminating cliché','Whataboutism','Bandwagon'
0,614_batch_2,If the logic of the anti-blockade protest appl...,0.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,NaN
1,605_batch_2,"Before we do the teacher, start this up your b...",0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN
2,380_batch_2,"42 days before I become my president, season t...",0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN
3,532_batch_2,LOOK SON \nDEMOCRATS DESTROYING STATUES OF RAC...,0.0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,NaN
4,454_batch_2,THINKING what they got\n\nSHE WIFE OR MY WHO?\...,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4386,128,Two kinds\n\nWomen\n\nMale,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4387,128,Two sex\n\nFemale\n\nmale,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4388,128,Two sexes\n\nwoman\n\nmale,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4389,128,Two sex\n\nWoman\n\nmale,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [ ]:
MODEL_NAME = 'roberta-base'
MAX_LEN = 128
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
#df = pd.read_csv('/content/drive/MyDrive/IITR/TheDataF.csv')
df2 = pd.read_csv('/content/drive/MyDrive/IITR/test_data.csv')
#df=df.drop(df.columns[0], axis=1)
#df=df.drop(df.columns[-1], axis=1)
#df=df.rename(columns={"text":"comment_text"})
df2=df2.drop("Unnamed: 0", axis=1)
df2=df2.rename(columns={"text":"comment_text"})
df_train = df
df_test = df2
#import json
#g = open('/content/drive/MyDrive/IITR/test_set_task1.txt', encoding='utf-8')
#final_test = pd.read_json(g, lines=True)
#final_test=final_test.rename(columns={"text":"comment_text"})
df2

,id,comment_text
0,705_batch_2,The Democrats New America\n
1,706_batch_2,WE ARE AT WAR!\n\nThere is a complex assault o...
2,710_batch_2,KILLED HIMSELF IN PRISON\n\nWON AN HONEST ELEC...
3,713_batch_2,I will never concede!\n\nNO WAY IN HELL BIDEN ...
4,715_batch_2,TRYING TO FIGURE OUT HOW BIDEN WON A RECORD LO...
...,...,...
195,761_batch_2,"Hi Joe, it's me...Cornpop\n"
196,762_batch_2,HE'S MAKING HIS LIST\n\nHE'S CHECKING IT TWICE...
197,764_batch_2,PA WHAT'S IRONY MEAN?\n\nIRONY IS WHEN THE LIB...
198,768_batch_2,Girlfriend Keeps Referring To Herself As 'Wife...


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
df['comment_text'] = df['comment_text'].str.lower()
df['comment_text'] = df['comment_text'].map(lambda com : clean_text(com))
df['comment_text'] = df['comment_text'].apply(cleanHtml)
df['comment_text'] = df['comment_text'].apply(cleanPunc)
df['comment_text'] = df['comment_text'].apply(keepAlpha)
df2['comment_text'] = df2['comment_text'].str.lower()
df2['comment_text'] = df2['comment_text'].map(lambda com : clean_text(com))
df2['comment_text'] = df2['comment_text'].apply(cleanHtml)
df2['comment_text'] = df2['comment_text'].apply(cleanPunc)
df2['comment_text'] = df2['comment_text'].apply(keepAlpha)
#final_test['comment_text'] = final_test['comment_text'].str.lower()
#final_test['comment_text'] = final_test['comment_text'].apply(cleanHtml)
#final_test['comment_text'] = final_test['comment_text'].apply(cleanPunc)
#final_test['comment_text'] = final_test['comment_text'].apply(keepAlpha)

stop_words = set(stopwords.words('english'))
#stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)
df['comment_text'] = df['comment_text'].apply(removeStopWords)
df2['comment_text'] = df2['comment_text'].apply(removeStopWords)
#final_test['comment_text'] = final_test['comment_text'].apply(removeStopWords)

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatization(sentence):
    lemmSentence = ""
    for word in sentence.split():
        lemm = lemmatizer.lemmatize(word)
        lemmSentence += lemm
        lemmSentence += " "
    lemmSentence = lemmSentence.strip()
    return lemmSentence
df['comment_text'] = df['comment_text'].apply(lemmatization)
df2['comment_text'] = df2['comment_text'].apply(lemmatization)
#final_test['comment_text'] = final_test['comment_text'].apply(lemmatization)

#final_test['comment_text'] = final_test['comment_text'].map(lambda com : clean_text(com))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
MODEL_NAME = 'roberta-base'
MAX_LEN = 128
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
def to_tokens(input_text, tokenizer):
    output = tokenizer.encode_plus(input_text, max_length=MAX_LEN, pad_to_max_length=True)
    return output

def select_field(features, field):
    return [feature[field] for feature in features]

In [ ]:
df_train=df
df_test=df2
tokenizer_output_train = df_train['comment_text'].apply(lambda x: to_tokens(x, tokenizer))
tokenizer_output_test = df_test['comment_text'].apply(lambda x: to_tokens(x, tokenizer))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
class CustomModel(TFRobertaPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super(CustomModel, self).__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels
        self.roberta = TFRobertaMainLayer(config, name="roberta")
        self.dropout_1 = tf.keras.layers.Dropout(0.3)
        self.classifier = tf.keras.layers.Dense(units=config.num_labels,
                                                name='classifier', 
                                                kernel_initializer=get_initializer(
                                                    config.initializer_range))

    def call(self, inputs, **kwargs):
        outputs = self.roberta(inputs, **kwargs)
        pooled_output = outputs[1]
        pooled_output = self.dropout_1(pooled_output, training=kwargs.get('training', False))
        logits = self.classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs

In [ ]:
def init_model(model_name):
    strategy = tf.distribute.experimental.TPUStrategy(resolver)
    with strategy.scope():
        config = RobertaConfig.from_pretrained(model_name, num_labels=2)
        model = CustomModel.from_pretrained(model_name)
        optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        metric = tf.keras.metrics.BinaryAccuracy('accuracy')
        model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    return model

In [ ]:
input_ids_train = np.array(select_field(tokenizer_output_train, 'input_ids'))
attention_masks_train = np.array(select_field(tokenizer_output_train, 'attention_mask'))

input_ids_test = np.array(select_field(tokenizer_output_test, 'input_ids'))
attention_masks_test = np.array(select_field(tokenizer_output_test, 'attention_mask'))

In [ ]:
BATCH_SIZE = 64
EPOCHS = 7
SPLITS = 2
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=2, verbose=2, 
                                              restore_best_weights=True)]
#model_output = np.zeros((df_sample_submission['target'].shape[0], 2))
skf = StratifiedKFold(n_splits=SPLITS, shuffle=True)

df_toxic = df.drop(['id', 'comment_text'], axis=1)
categories = list(df_toxic.columns.values)
for category in categories:


  print('\n\n ... Processing {}'.format(category))
  print("\n")

  model_output = np.zeros((df2.shape[0], 2))
  X, y = input_ids_train, df_train[category].values.reshape(-1, 1)
  skf.get_n_splits(X, y)
  for i, (train_index, test_index) in enumerate(skf.split(X, y)):
      
      #saved_model = open('Saved_Model'+ category + '_' + str(i) + '.sav', 'wb')


      X_train, attention_masks_train_stratified = X[train_index], attention_masks_train[train_index]
      X_test, attention_masks_test_stratified =  X[test_index], attention_masks_train[test_index]
      y_train, y_test = tf.keras.utils.to_categorical(y[train_index]), tf.keras.utils.to_categorical(y[test_index])
      X_train = X_train[:-divmod(X_train.shape[0], BATCH_SIZE)[1]]
      attention_masks_train_stratified = attention_masks_train_stratified[:-divmod(attention_masks_train_stratified.shape[0], 
                                                                                  BATCH_SIZE)[1]]
      y_train = y_train[:-divmod(y_train.shape[0], BATCH_SIZE)[1]]
      model = init_model(MODEL_NAME)
      model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
      model.fit([X_train, attention_masks_train_stratified], y_train, 
                validation_data=([X_test, attention_masks_test_stratified], y_test), 
                batch_size=BATCH_SIZE, epochs=EPOCHS)
      
      
      #filename = '/content/drive/MyDrive/saved_models/'+ category.replace("/",",").replace("'", "") + '.h5'
      #model.save(filename)  # creates a HDF5 file
      #pickle.dump(model,saved_model)
    # returns a compiled model
    # identical to the previous one
#   model = load_model('my_model.h5')

      
      model_output = model_output+model.predict([input_ids_test, attention_masks_test])[0]
      #df2[category] = model_output
      #df2[category] = np.argmax(model_output, axis=1).flatten()
      del model
      gc.collect()
      print('='*22 + ' Split ' + str(i+1) + ' finished ' + '='*22)
  model_output /= SPLITS
  df2[category] = np.argmax(model_output, axis=1).flatten()

In [ ]:
df2["labels"]=""
print(df2.shape)
for row in df2.index:
    dict2=[]
    for category in categories:
        if (df2[category][row]==1):
            dict2.append(category[1:-1])
            #print("hello")
            #df2["labels"][row]+=pil
    dict2.sort()
    df2['labels'][row]=dict2
#df2["labels"]=df2["labels"].str.rstrip(',')
#df2["labels"]+=']'
submission = df2[['id','comment_text','labels']]
submission.head()
out = submission.to_json(orient='records', indent=4)
with open('temp.txt', 'w') as f:
    f.write(out)

In [ ]:
for category in categories:
  print(df2[category].value_counts())